<a href="https://colab.research.google.com/github/SuYenTing/d4sg_cipas/blob/main/cipas_ckip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CKIP實體辨識

2022/02/06 蘇彥庭

參考文件:
* [Github: ckiptagger](https://github.com/ckiplab/ckiptagger)
* [基於詞性組合規則結合維基百科
進行中文命名實體辨識與消歧義](https://www.lac.org.tw/sites/default/files/field_files/publish/attach241_1.pdf)

In [ ]:
# 掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 安裝套件
!pip install -U ckiptagger

In [ ]:
# 下載模型
!pip install --upgrade --no-cache-dir gdown  # 需升級gdown套件避免下載模型時報錯
from ckiptagger import data_utils
data_utils.download_data_gdown("/content/")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.3.0-py3-none-any.whl size=14412 sha256=6addab467bf8f15241c7b40177ff67c78dad0bb837b0908f1f9372e6a17f7e2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-d8diqlnh/wheels/fd/ce/f8/389eafb78bce55ea78740dfcafc3c9da6f5e70d25c0377610d
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.1
    Uninstalling gdown-4.2.1:
      Successfully uninstalled gdown-4.2.1


Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:10<00:00, 177MB/s]


In [ ]:
# 載入套件
import numpy as np
import pandas as pd
import pickle
import re
from tqdm import tqdm
from ckiptagger import construct_dictionary, WS, POS, NER
# from gensim.models import Word2Vec

In [ ]:
# 讀取資料
cipasWebContentData = pd.read_csv('/content/drive/MyDrive/[不當黨產]/Data/爬蟲資料/cipasWebContentData.csv')
pdfData = pd.read_csv('/content/drive/MyDrive/[不當黨產]/Data/爬蟲資料/pdfData.csv')

In [ ]:
# # 取出調查報告文章資料(PDF檔案內容)
# reportData = pdfData[(pdfData['source'] == 'news') & (pdfData['attachFileNames'].str.contains('調查報告'))]
# reportData = reportData[~(reportData['attachFileNames'].str.contains('PPT|簡報|投影片'))]
# reportData = reportData[['source', 'id', 'attachFileNames', 'content']].rename(columns={'attachFileNames': 'articleTitle', 'content': 'articleContent'})

# 取出史料故事文章資料(網站內容)
storiesData = cipasWebContentData[cipasWebContentData['source'] == 'stories'][['source', 'id', 'articleTitle', 'articleContent']]

# 合併資料
# docData = pd.concat([reportData, storiesData]).reset_index(drop=True)
docData = storiesData.reset_index(drop=True)

In [ ]:
# 載入模型
wsModel = WS("./data", disable_cuda=False)  # disable_cuda=False: use gpu
posModel = POS("./data", disable_cuda=False)
nerModel = NER("./data", disable_cuda=False)

/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)
/usr/local/lib/python3.7/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class

In [ ]:
 # 讀取Google雲端sheet的自定義斷詞
customWords = []

sheetDict = {
    '不當黨產相關名詞詞庫_黨產會斷詞庫': 'https://docs.google.com/spreadsheets/d/1ayhAdNJ_Tc9-6ptEkCZSQe-GJ0obpZoAsHvXIUVFOK4/edit#gid=1370388179',
    '不當黨產相關名詞詞庫_資料英雄斷詞': 'https://docs.google.com/spreadsheets/d/1ayhAdNJ_Tc9-6ptEkCZSQe-GJ0obpZoAsHvXIUVFOK4/edit#gid=725229876',
    '不當黨產相關名詞詞庫_直式範本': 'https://docs.google.com/spreadsheets/d/1ayhAdNJ_Tc9-6ptEkCZSQe-GJ0obpZoAsHvXIUVFOK4/edit#gid=1583708380',
}

for sheetName in sheetDict:

    sheetUrl = sheetDict[sheetName]
    sheetUrl = sheetUrl.replace('/edit#gid=', '/export?format=csv&gid=')

    if sheetName == '不當黨產相關名詞詞庫_黨產會斷詞庫':
        iCustomWords = pd.read_csv(sheetUrl, header=None)
        iCustomWords = iCustomWords.iloc[:, 0:2]

    elif sheetName == '不當黨產相關名詞詞庫_資料英雄斷詞':
        iCustomWords = pd.read_csv(sheetUrl, header=None)
        iCustomWords = iCustomWords.iloc[:, 0:1]

    elif sheetName == '不當黨產相關名詞詞庫_直式範本':
        iCustomWords = pd.read_csv(sheetUrl)

    iCustomWords = iCustomWords.to_numpy().flatten()
    iCustomWords = iCustomWords[~pd.isnull(iCustomWords)]
    iCustomWords = list(set(iCustomWords))
    customWords = customWords + iCustomWords

# 刪除重複的詞
customWords = list(set(customWords))

# 建立自定義詞字典
dictionary = {elem: 1 for elem in customWords}
dictionary = construct_dictionary(dictionary)

In [ ]:
# 清洗資料函數
def cleanText(text):
    
    # 移除數字, 空格(含跳行), 英文句點, 英文逗點
    # text = re.sub('\d|\s|\.|\,', '', text)

    # 移除空格(含跳行)
    text = re.sub('\s', '', text)

    # 移除 資料來源 / 本文參考 / 全文參考 / 引自 (含)以後的字元
    matchSite = re.search('資料來源|本文參考|全文參考|引自', text)
    if matchSite:
        text = text[:matchSite.span()[0]]

    return text

In [ ]:
# 建立儲存表
wordSentenceList = []  # 儲存各句斷詞後結果
entityList = []  # 儲存實體辨識結果
posList = []  # 儲存詞性結果
personJobList = []  # 儲存姓+職稱+名(利用詞性Nb+Na+Nb組合)

# 篩選詞性清單
targetPos = ['Na', 'Nb', 'Nc']
# 篩選實體清單
targetEntity = ['ORG', 'PERSON', 'EVENT', 'FAC', 'GPE', 'LOC', 'WORK_OF_ART']

# 迴圈文本資料
# iDoc = 200
for iDoc in tqdm(range(len(docData))):

    # 讀取文本並清洗資料
    doc = docData['articleContent'][iDoc]
    doc = cleanText(doc)

    # 以句號作為句子分隔
    doc = doc.split('。')

    # 執行CKIP斷詞
    wsModelResult = wsModel(
        doc,
        sentence_segmentation=True,  # To consider delimiters
        segment_delimiter_set={",", "。", ":", "?", "!", ";"},  # This is the defualt set of delimiters
        coerce_dictionary=dictionary,  # words in this dictionary are forced
        )
    # 執行CKIP詞性標註
    posModelResult = posModel(wsModelResult)
    # 執行CKIP實體辨識
    entityModelResult = nerModel(wsModelResult, posModelResult)

    # 儲存斷詞結果
    for word_sentence in wsModelResult:
        wordSentenceList.append(word_sentence)

    # 儲存詞性結果
    for i in range(len(wsModelResult)):

        wordSetence = wsModelResult[i]
        posSentence = posModelResult[i]
        assert len(wordSetence) == len(posSentence)

        # 整理目標詞性
        for word, pos in zip(wordSetence, posSentence):
            if pos in targetPos:
                if len(word) > 1:  # 詞至少要有一個字(單字不成詞)
                    posList.append([word, pos])

        # 整理姓+職稱+名(利用詞性Nb+Na+Nb組合)
        for i in range(len(posSentence)):
            if posSentence[i:(i+3)] == ['Nb', 'Na', 'Nb']:
                if (len(wordSetence[i]) <= 2) and (len(wordSetence[i+2]) <= 2):  # 姓和名只能2個字以下
                    personJobList.append(wordSetence[i:(i+3)])
                    
    # 儲存實體辨識結果
    for res in entityModelResult:
        for elem in res:
            if elem[2] in targetEntity:  # 判斷是否在想要關注的實體標示類別內
                if len(elem[3]) > 1:  # 詞至少要有一個字(單字不成詞)
                    entityList.append([elem[3], elem[2]])

    # # 印出CKIP結果
    # def print_word_pos_sentence(word_sentence, pos_sentence):
    #     assert len(word_sentence) == len(pos_sentence)
    #     for word, pos in zip(word_sentence, pos_sentence):
    #         print(f"{word}({pos})", end="\u3000")
    #     print()
    #     return

    # for i, sentence in enumerate(wsModelResult):
    #     print()
    #     print(f"'{sentence}'")
    #     print_word_pos_sentence(wsModelResult[i],  posModelResult[i])
    #     for entity in sorted(entityModelResult[i]):
    #         print(entity)

100%|██████████| 298/298 [03:24<00:00,  1.45it/s]


In [ ]:
# 整理輸出結果
entityDf = pd.DataFrame(entityList, columns=['word', 'type'])
posDf = pd.DataFrame(posList, columns=['word', 'type'])
personJobDf = pd.DataFrame(personJobList, columns=['lastName', 'jobTitle', 'firstName'])
personJobDf['person'] = personJobDf['lastName'] + personJobDf['jobTitle'] + personJobDf['firstName']
personJobDf['word'] = personJobDf['lastName'] + personJobDf['firstName']

In [ ]:
# 計算詞頻數量
entityDf = entityDf.groupby(['word', 'type']).size().reset_index(name='counts').sort_values(['counts'], ascending=False).reset_index(drop=True)
posDf = posDf.groupby(['word', 'type']).size().reset_index(name='counts').sort_values(['counts'], ascending=False).reset_index(drop=True)
personJobDf = personJobDf.groupby(['lastName', 'jobTitle', 'firstName', 'person', 'word']).size().reset_index(name='counts').sort_values(['counts'], ascending=False).reset_index(drop=True)

In [ ]:
# 新增欄位判斷是否已在斷詞辭典內
entityDf['inDict'] = entityDf['word'].isin(customWords).apply(lambda x: '1' if x else '')
posDf['inDict'] = posDf['word'].isin(customWords).apply(lambda x: '1' if x else '')
personJobDf['inDict'] = personJobDf['person'].isin(customWords).apply(lambda x: '1' if x else '')

In [ ]:
# 輸出結果
entityDf.to_csv('entityDf.csv', index=False, encoding='utf-8-sig')
posDf.to_csv('posDf.csv', index=False, encoding='utf-8-sig') 
personJobDf.to_csv('personJobDf.csv', index=False, encoding='utf-8-sig') 

In [ ]:
# 確認那些字典自定義詞沒有被捕捉到
catchWordLists = entityDf['word'][entityDf['inDict'] == '1'].tolist() + \
                 posDf['word'][posDf['inDict'] == '1'].tolist() + \
                 personJobDf['person'][personJobDf['inDict'] == '1'].tolist()
notCatchWordLists = [elem for elem in customWords if elem not in catchWordLists]

In [ ]:
# 確認實體辨識結果是否皆在詞性標註結果內
orgAndPerson = entityDf[entityDf['type'].isin(['ORG', 'PERSON'])]['word'].tolist()
orgAndPersonNotInPos = [elem for elem in orgAndPerson if elem not in posDf[posDf['type'].isin(['Na', 'Nb', 'Nc'])]['word'].tolist()]

In [ ]:
# 將實體辨識結果併入詞性內
posDf['ORG'] = posDf['word'].isin(entityDf[entityDf['type'] == 'ORG']['word']).apply(lambda x: 'ORG' if x else '')
posDf['PERSON'] = posDf['word'].isin(entityDf[entityDf['type'] == 'PERSON']['word']).apply(lambda x: 'PERSON' if x else '')

In [ ]:
posDf.to_csv('posDf.csv', index=False, encoding='utf-8-sig') 